In [6]:
from math import *

def gaussian_f(mu, sigma2, x):
    return ((1/sqrt(2*pi*sigma2)*(exp(-.5*(x-mu)**2/sigma2))))

             

gaussian_f(10,4,8)

0.12098536225957168

In [12]:
# Write a program to update your mean and variance
# when given the mean and variance of your belief
# and the mean and variance of your measurement.
# This program will update the parameters of your
# belief function.

#1D Kalman Filter Components
#Update is our measurement model
#This is our Bayes Rule
def update(mean1, var1, mean2, var2):
    new_mean = (mean1*var2 + mean2*var1)/(var1 + var2)
    new_var = (var1**-1 + var2**-1)**-1
    return [new_mean, new_var]

#Predict is our motion model
#This is our convolution
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    
    return [new_mean, new_var]

print(update(10.,8.,13., 2.))
print(update(10.,8.,13., 2.))

[12.4, 1.6]


In [23]:
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

for n in range(len(measurements)):
    #initialize with our values that we start with and run an update/measure 
    [mu, sig] = update(mu, sig, measurements[n], measurement_sig)
    print('update:', [mu,sig])
    [mu,sig] = predict(mu, sig, motion[n], motion_sig)
    print('predict: ', [mu, sig])

update: [4.998000799680128, 3.9984006397441023]
predict:  [5.998000799680128, 5.998400639744102]
update: [5.999200191953932, 2.399744061425258]
predict:  [6.999200191953932, 4.399744061425258]
update: [6.999619127420922, 2.0951800575117594]
predict:  [8.999619127420921, 4.09518005751176]
update: [8.999811802788143, 2.0235152416216957]
predict:  [9.999811802788143, 4.023515241621696]
update: [9.999906177177365, 2.0058615808441944]
predict:  [10.999906177177365, 4.005861580844194]


In [32]:
# Fill in the matrices P, F, H, R and I at the bottom
#
# This question requires NO CODING, just fill in the 
# matrices where indicated. Please do not delete or modify
# any provided code OR comments. Good luck!

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            print("Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            print("Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print( self.value[i])
        print( ' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            print("Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            print("Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            print("Matrices must be m*n and n*p to multiply")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    print("Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


########################################

def filter(x, P):
    for n in range(len(measurements)):
        
        # prediction
        x = (F * x) + u
        P = F * P * F.transpose()
        
        # measurement update
        Z = matrix([measurements[n]])
        y = Z.transpose() - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * y)
        P = (I - (K * H)) * P
    
    print( 'x= ')
    x.show()
    print( 'P= ')
    P.show()

########################################

print( "### 4-dimensional example ###")

measurements = [[5., 10.], [6., 8.], [7., 6.], [8., 4.], [9., 2.], [10., 0.]]
initial_xy = [4., 12.]

measurements = [[1., 4.], [6., 0.], [11., -4.], [16., -8.]]
# initial_xy = [-4., 8.]

measurements = [[1., 17.], [1., 15.], [1., 13.], [1., 11.]]
# initial_xy = [1., 19.]

dt = 0.1

x = matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]]) # initial state (location and velocity)
u = matrix([[0.], [0.], [0.], [0.]]) # external motion

#### DO NOT MODIFY ANYTHING ABOVE HERE ####
#### fill this in, remember to use the matrix() function!: ####
# initial uncertainty: 0 for positions x and y, 1000 for the two velocities
P =  matrix([[0.,0.,0.,0.],
             [0.,0.,0.,0.],
             [0.,0.,1000.,0.],
             [0.,0.,0.,1000.]])
# next state function: generalize the 2d version to 4d
F = matrix([[1.,0.,dt,0.],
             [0.,0.,0.,dt],
             [0.,0.,1.,0.],
             [0.,0.,0.,1.]])
# measurement function: reflect the fact that we observe x and y but not the two velocities
H =  matrix([[1.,0.,0.,0.],
             [0.,1.,0.,0.]])
# measurement uncertainty: use 2x2 matrix with 0.1 as main diagonal
R =  matrix([[0.1,0.], [0.,0.1]])
# 4d identity matrix
I = matrix([[1.,0.,0.,0.],
             [0.,1.,0.,0.],
             [0.,0.,1.,0.],
             [0.,0.,0.,1.]])

###### DO NOT MODIFY ANYTHING HERE #######

filter(x, P)



### 4-dimensional example ###
x= 
[0.0013328890369912472]
[13.965087281795507]
[-9.996667777407497]
[139.65087281795505]
 
P= 
[0.05331556147950691, 0.0, 0.13328890369876803, 0.0]
[0.0, 0.024937655860349166, 0.0, 0.24937655860349162]
[0.1332889036987679, 0.0, 0.33322225924692717, 0.0]
[0.0, 0.24937655860349164, 0.0, 2.4937655860349164]
 


In [47]:
 # Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
#by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.

from math import *
import random



landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            print('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            print('Y coordinate out of bound')

        if new_orientation < 0 or new_orientation >= 2 * pi:
            print('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            print('Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))



####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.

myrobot = robot()

print('Initialize robot: ', myrobot.set(30.0, 50.0, pi/2))
myrobot.set_noise(5.0, 0.1, 5.0)
myrobot = myrobot.move(-pi/2, 15.0)
print('Robot senses: ', myrobot.sense())
myrobot = myrobot.move(-pi/2, 10.0)
print('Robot senses: ', myrobot.sense())




Initialize robot:  None
Robot senses:  [33.54821699123512, 48.94685591847801, 41.236981426412775, 48.61140764950131]
Robot senses:  [30.490090127214266, 50.63434931048048, 47.408962203286954, 59.101688759506715]


In [79]:
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
T = 100
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)
print(eval(myrobot, p))

#this is a time range
for t in range(T):
    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()
    
    
    p2 = []
    for i in range(N):
        p2.append(p[i].move(0.1, 5.0))
    p = p2

    w = []
    for i in range(N):
        w.append(p[i].measurement_prob(Z))
    #print(w[:10])

    p3 = []

    #Randomly grab a particle with this index 
    index = int(random.random() * N)
    beta = 0.0
    mw = max(w)
    for i in range(N):
        beta += random.random() * 2.0 * mw
        while beta > w[index]:
            beta -= w[index]
            index = (index + 1) % N

        p3.append(p[index])

    p = p3
    
    print(eval(myrobot, p))
#print(p[10:20])
    

38.48206095013001
4.897843645839195
4.0324582226869
3.125744169717206
4.144237303037476
5.626477022764932
5.830071465615684
4.029912721027543
3.5612688794739316
3.4007820449319386
3.143566675589114
2.804462617311283
2.6596675084147683
1.3922062517415374
1.6794489313562635
1.8594771593003332
1.941403005763276
1.8394331382399147
1.7524777708448804
1.7095533613955995
1.659687255753726
1.678348107442825
1.6914786325433246
1.7240965418312446
1.7709113111497619
1.7903004943027203
1.8226707335158832
1.7930277183250787
1.8183627548595833
1.7629628286319297
1.776653363842811
1.7339483034542738
1.776262352105813
1.8594299930129012
1.8670228829111766
1.864282401391687
1.8437494009375217
1.8069158156881893
1.7443707989112147
1.731096494925376
1.733056704520869
1.7694639989678538
1.7705514332685275
1.7890812732954484
1.8434212992855548
1.9187091226593043
1.8777502008184481
1.88038544071054
1.852237117093917
1.8235002863428151
1.8090359701230663
1.7905761809666
1.7990683679025592
1.7903935830484772
